In [146]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# GENRAL OBSERVATION

- Also in MetaRNN they have missing data and they were dealing with it with BPCAfill
- We need to standardize feature
did they use information such as 

In [226]:
df = pd.read_csv('output.hg38_multianno.csv', engine="python",  on_bad_lines="warn")
df = df.replace(".", pd.NA)

In [227]:
df.head()
print(df.columns.tolist())

['Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.refGene', 'Gene.refGene', 'GeneDetail.refGene', 'ExonicFunc.refGene', 'AAChange.refGene', 'SIFT_score', 'SIFT_converted_rankscore', 'SIFT_pred', 'SIFT4G_score', 'SIFT4G_converted_rankscore', 'SIFT4G_pred', 'Polyphen2_HDIV_score', 'Polyphen2_HDIV_rankscore', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_score', 'Polyphen2_HVAR_rankscore', 'Polyphen2_HVAR_pred', 'LRT_score', 'LRT_converted_rankscore', 'LRT_pred', 'LRT_Omega', 'MutationTaster_score', 'MutationTaster_converted_rankscore', 'MutationTaster_pred', 'MutationAssessor_score', 'MutationAssessor_rankscore', 'MutationAssessor_pred', 'FATHMM_score', 'FATHMM_converted_rankscore', 'FATHMM_pred', 'PROVEAN_score', 'PROVEAN_converted_rankscore', 'PROVEAN_pred', 'VEST4_score', 'VEST4_rankscore', 'MetaSVM_score', 'MetaSVM_rankscore', 'MetaSVM_pred', 'MetaLR_score', 'MetaLR_rankscore', 'MetaLR_pred', 'Reliability_index', 'MetaRNN_score', 'MetaRNN_rankscore', 'MetaRNN_pred', 'M-CAP_score', 'M-CAP_rankscor

### MetaRNN input and our table
- SIFT (we have SIFT score, pred and others)
- Polyphen2 HDIV (score and pred)
- Polyphen2 HVAR (score and pred)
- Mutation Assessor
- PROVEAN
- Vest4
- M-CAP 
- MutPred
- MVP
- PrimateAI
- DEODEN
- CADD
...
- MutationTaster
- all meta meta RNN...

In [255]:
input_metarnn = ['SIFT_score', 'Polyphen2_HDIV_score', 'Polyphen2_HVAR_score', 'MutationAssessor_score', 'PROVEAN_score', 'VEST4_score',
                 'M-CAP_score', 'MutPred_score', 'MVP_score', 'PrimateAI_score', 'DEOGEN2_score', 'CADD_raw', 'fathmm-MKL_coding_score', 'Eigen-raw_coding',
                 'GERP++_RS_rankscore', 'phyloP100way_vertebrate', 'phastCons470way_mammalian', 'phyloP17way_primate', 'phastCons100way_vertebrate', 
                 'phastCons17way_primate', 'SiPhy_29way_pi', 'MutationTaster_score', 'GenoCanyon_score']
removed = ['phyloP470way_mammalian''1000g2015aug_all']
input_metarnn_important_features = ['VEST4_score', 'MutPred_score', 'M-CAP_score'] 
input_wp12_genomics = ['REVEL_score', 'SIFT_score', 'MutationTaster_score', 'PROVEAN_score'] # , 'AF_popmax'] 

## IMPORTANT 
### BASICALLY WE MISS ALL INFORMATION ON ALLELE FREQUENCY
- Here miss gnomac and ExAC features which are important accordingly to metaRNN SHAP analysis (but also are strongly related so prob just one of this is enough)
- Not sure 1000g2015aug_all is 1000GP...
- AFPOPMAX is gnomad

In [256]:
[col for col in df.columns if "af" in col.lower()]

['BayesDel_addAF_score',
 'BayesDel_addAF_rankscore',
 'BayesDel_addAF_pred',
 'BayesDel_noAF_score',
 'BayesDel_noAF_rankscore',
 'BayesDel_noAF_pred',
 'AF',
 'AF_popmax',
 'AF_male',
 'AF_female',
 'AF_raw',
 'AF_afr',
 'AF_sas',
 'AF_amr',
 'AF_eas',
 'AF_nfe',
 'AF_fin',
 'AF_asj',
 'AF_oth',
 'non_topmed_AF_popmax',
 'non_neuro_AF_popmax',
 'non_cancer_AF_popmax',
 'controls_AF_popmax']

In [257]:
df['SIFT_score'].value_counts(dropna=False)

SIFT_score
NaN      2173
0.0       502
0.001     236
0.002      88
0.003      46
         ... 
0.052       1
0.137       1
0.237       1
0.086       1
0.13        1
Name: count, Length: 318, dtype: int64

which column has more nan?
- phyloP470way_mammalian 
- 1000g2015aug_all

In [258]:
df[input_wp12_genomics].isna().sum().sort_values(ascending=False)

REVEL_score             2225
SIFT_score              2173
PROVEAN_score           2173
MutationTaster_score    1856
dtype: int64

## Are all the input there if there is the MetaRNN output?

In [259]:
df_metarnn = df[['Chr', 'Start', 'End', 'Ref', 'Alt'] + input_wp12_genomics + ['MetaRNN_score']]

In [260]:
df_metarnn

,Chr,Start,End,Ref,Alt,REVEL_score,SIFT_score,MutationTaster_score,PROVEAN_score,MetaRNN_score
0,19,11089263,11089263,C,G,NaN,NaN,NaN,NaN,NaN
1,19,11089281,11089281,G,T,NaN,NaN,NaN,NaN,NaN
2,19,11089309,11089311,CAG,C,NaN,NaN,NaN,NaN,NaN
3,19,11089318,11089319,AC,A,NaN,NaN,NaN,NaN,NaN
4,19,11089318,11089458,ACGGGTTAAAAAGCCGATGTCACATCGGCCGTTCGAAACTCCTCCT...,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3701,19,11133598,11133598,G,C,NaN,NaN,NaN,NaN,NaN
3702,19,11133666,11133666,C,G,NaN,NaN,NaN,NaN,NaN
3703,19,11133681,11133681,C,T,NaN,NaN,NaN,NaN,NaN
3704,19,11133682,11133682,G,T,NaN,NaN,NaN,NaN,NaN


In [261]:
df_metarnn = df_metarnn.dropna(subset=["MetaRNN_score"])


In [262]:
df_metarnn

,Chr,Start,End,Ref,Alt,REVEL_score,SIFT_score,MutationTaster_score,PROVEAN_score,MetaRNN_score
123,19,11089549,11089549,A,C,0.638,0.0,1,-0.69,0.8686191
124,19,11089549,11089549,A,G,0.632,0.0,1,-1.0,0.88573956
125,19,11089549,11089549,A,T,0.638,0.0,1,-0.69,0.8685801
126,19,11089550,11089550,T,C,0.621,0.0,1,-0.82,0.8593923
128,19,11089551,11089551,G,A,0.613,0.0,1,-1.08,0.88374764
...,...,...,...,...,...,...,...,...,...,...
3591,19,11131334,11131334,C,T,NaN,0.0,1,-0.25,0.07244429
3595,19,11131457,11131457,G,A,NaN,0.0,1,0.32,0.0017805696
3596,19,11131485,11131485,G,T,NaN,0.0,1,-0.02,0.12277198
3597,19,11131539,11131539,G,A,NaN,NaN,1,NaN,0.0042452216


In [263]:
df_no_nan = df_metarnn.dropna()

In [264]:
df_no_nan

,Chr,Start,End,Ref,Alt,REVEL_score,SIFT_score,MutationTaster_score,PROVEAN_score,MetaRNN_score
123,19,11089549,11089549,A,C,0.638,0.0,1,-0.69,0.8686191
124,19,11089549,11089549,A,G,0.632,0.0,1,-1.0,0.88573956
125,19,11089549,11089549,A,T,0.638,0.0,1,-0.69,0.8685801
126,19,11089550,11089550,T,C,0.621,0.0,1,-0.82,0.8593923
128,19,11089551,11089551,G,A,0.613,0.0,1,-1.08,0.88374764
...,...,...,...,...,...,...,...,...,...,...
3577,19,11131294,11131294,G,C,0.521,0.002,1.000,-2.31,0.64217454
3578,19,11131296,11131296,C,A,0.352,0.027,0.69291,-0.72,0.15382192
3579,19,11131296,11131296,C,G,0.367,0.063,0.748134,-1.2,0.135245
3580,19,11131299,11131299,G,A,0.737,0.006,1,-2.53,0.43771297
